Author: Adafaly Matthieu </br>

# Importing libraries


In [ ]:
import plotly.express as px
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.io as pio
from scipy.stats import norm
import scipy.stats as stats
from sklearn.model_selection import train_test_split
import calendar
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.stats.diagnostic import acorr_ljungbox
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import adfuller


# Data


In [ ]:
df = pd.read_pickle("Data/pollution_rennes_filter.pkl")
print("dataframe loaded")

In [ ]:
display(df)

In [ ]:
df_stationnary = df.loc[(df['sensor_type'] == 'fixedGps') & (df['PM_2.5'].notna())]
df_mobile = df.loc[(df['sensor_type'] == 'mobileGps') & (df['PM_2.5'].notna())]

## Data visualization

In [ ]:
# Calculate the hourly average PM₂.₅ per sensor
mean_values = df_stationnary.groupby(['hour', 'sensor_name'])['PM_2.5'].mean().reset_index()

# Remove a specific station if needed
mean_values = mean_values[mean_values['sensor_name'] != 'standalone-LOPY-AQ05']

# Set the figure size
plt.figure(figsize=(12, 6))

# Plot the lines using Seaborn
sns.lineplot(data=mean_values, x='hour', y='PM_2.5', hue='sensor_name')

# Add title and labels
plt.title("Average PM₂.₅ Concentration by Hour and Sensor")
plt.xlabel("Time of Day (Hour)")
plt.ylabel("PM₂.₅ Concentration (µg/m³)")
plt.legend(title="Station", bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
# Définir l'ordre des jours de la semaine
week_days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']

# Copier et ordonner
df = df_stationnary.copy()
df['day_week'] = pd.Categorical(df['day_week'], 
                                 categories=week_days_order, 
                                 ordered=True)

# Calculer les moyennes sur le bon DataFrame
mean_values = df.groupby(['day_week', 'sensor_name'], observed=True)['PM_2.5'].mean().reset_index()

# Tracer
plt.figure(figsize=(12, 6))
sns.lineplot(data=mean_values, x='day_week', y='PM_2.5', hue='sensor_name', marker='o')

# Ajouts esthétiques
plt.title("Average PM₂.₅ Concentration by Day of the Week and Sensor")
plt.xlabel("Day of the Week")
plt.ylabel("PM₂.₅ Concentration (µg/m³)")
plt.legend(title="Sensor", bbox_to_anchor=(1.05, 0.5), loc='center left')
plt.tight_layout()
plt.show()


In [ ]:
pio.renderers.default = 'notebook'  # ou 'iframe', 'notebook_connected'
df_box = df.copy()
df_box['sensor_name'] = df_box.index.get_level_values('sensor_name')

fig = px.box(df_box, 
             x="sensor_name", 
             y="PM_2.5", 
             title="Distribution of pollution value by sensor", 
             labels={"PM_2.5": "Pollution value (µg/m³)", "sensor_name": "Sensor name"})

# Update the margins to recenter the box
fig.update_layout(
    margin=dict(l=40, r=40, t=40, b=40),  # Adjust the left, right, top, bottom margins
    boxmode='group',  # Ensure that the boxes do not overlap
    yaxis=dict(
        range=[df['PM_2.5'].quantile(0.05), df['PM_2.5'].quantile(0.95)]  # Limit the y-axis to the 5-95% of the data
    )
)

# Display the graph
fig.show()

In [ ]:
lissage=30
global_mean = df.groupby(
    df.index.get_level_values('measure_date').floor('D')
)['PM_2.5'].mean().reset_index()

global_mean.columns = ['date', 'mean_v']


# Adding the smoothed mean
global_mean['smoothed_mean'] = global_mean['mean_v'].rolling(window=lissage, center=True).mean()

# Interactive plot of the global mean
fig = px.line(
    global_mean,
    x='date',
    y='mean_v',
    markers=True,
    title="Daily Mean PM₂.₅ Concentration",
    labels={
        'mean_v': 'Concentration (µg/m³)',
        'date': 'Date'
    }
)

# Adding the smoothed line (black line)
fig.add_scatter(
    x=global_mean['date'],
    y=global_mean['smoothed_mean'],
    mode='lines',
    name=f'Smoothed mean ({lissage} days)',
    line=dict(color='black', width=3)
)
fig.update_traces(line_color='darkorange', line_width=3, selector=dict(name=None))  # main line
fig.update_layout(hovermode='x unified')

fig.show()

### Autocorrelation


In [ ]:
df=df.reset_index()
df_IRISA = df.loc[(df['sensor_name'] == 'standalone-LOPY-AQ03') & (df['PM_2.5'].notna())]

In [ ]:
plot_acf(df_IRISA["PM_2.5"].dropna(), lags=4230)  
plt.title("Autocorrelation of PM2.5 Concentration")
plt.xlabel("Lag")
plt.ylabel("Autocorrelation Coefficient")
plt.grid(True)
plt.show()


### PM₂.₅ Data Distribution Across Different Variables


In [ ]:
months_labels = ['January', 'February', 'March', 'April', 'May', 'June',
                 'July', 'August', 'September', 'October', 'November', 'December']


fig, axes = plt.subplots(4, 3, figsize=(15, 12))
axes = axes.flatten()

# Plot histograms + KDE + normal distribution
for i in range(1, 13):
    ax = axes[i - 1]
    data = df_IRISA[df_IRISA['month'] == i]['PM_2.5'].dropna()

    if len(data) < 2:
        ax.set_title(f"{months_labels[i - 1]} (Not enough data)")
        continue

    # Histogram + KDE with Seaborn
    sns.histplot(data, bins=200, kde=True, ax=ax, color='orange', stat="density", edgecolor=None)

    # Normal distribution curve
    mu = data.mean()
    sigma = data.std()
    x = np.linspace(0, 40, 500)
    y = norm.pdf(x, mu, sigma)
    ax.plot(x, y, color='red', linestyle='--', linewidth=2, label='Normal')

    ax.set_title(months_labels[i - 1])
    ax.set_xlim(0, 40)
    ax.set_xlabel("PM₂.₅ (µg/m³)")
    ax.set_ylabel("Density")
    ax.legend()

# Remove empty axes
for j in range(len(months_labels), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("PM₂.₅ Distribution by Month with Normal Curve", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
data = df['PM_2.5'].dropna()
data_log = np.log(data + 1)

months_labels = ['January', 'February', 'March', 'April', 'May', 'June',
                 'July', 'August', 'September', 'October', 'November', 'December']

# Create a figure with 4 rows and 3 columns
fig, axes = plt.subplots(4, 3, figsize=(15, 12))
axes = axes.flatten()

# Plot histograms + KDE + normal distribution
for i in range(1, 13):
    ax = axes[i - 1]
    data = df_IRISA[df_IRISA['month'] == i]['PM_2.5'].dropna()
    data_log = np.log(data + 1)
    if len(data_log) < 2:
        ax.set_title(f"{months_labels[i - 1]} (Not enough data)")
        continue

    # Histogram + KDE with Seaborn
    sns.histplot(data_log, bins=200, kde=True, ax=ax, color='orange', stat="density", edgecolor=None)

    # Normal distribution
    mu = data_log.mean()
    sigma = data_log.std()
    x = np.linspace(0, 40, 500)
    y = norm.pdf(x, mu, sigma)
    ax.plot(x, y, color='red', linestyle='--', linewidth=2, label='Normal')

    ax.set_title(months_labels[i - 1])
    ax.set_xlim(0, 5)
    ax.set_xlabel("PM₂.₅ (µg/m³)")
    ax.set_ylabel("Density")
    ax.legend()

# Remove empty axes
for j in range(len(months_labels), len(axes)):
    fig.delaxes(axes[j])

fig.suptitle("PM₂.₅ Distribution by Month with Normal Curve", fontsize=16)
plt.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()


In [ ]:
df['PM_2.5'] = pd.to_numeric(df['PM_2.5'], errors='coerce')

In [ ]:
data = df['PM_2.5']
stats.probplot(data, dist="norm", plot=plt)
plt.title("Q-Q plot for log(PM2.5)")
plt.grid()
plt.show()


### Aggregate by day

In [ ]:
df['date'] = pd.to_datetime(df['date'])  # Ensure datetime format
df_day = df.groupby(df['date'].dt.floor('D'))['PM_2.5'].mean().reset_index()
df_day['date'] = df_day['date'].dt.tz_localize(None)  # Remove timezone if present
display(df_day)

In [ ]:
df_day['year_month'] = df_day['date'].dt.to_period('M')
count_per_month = df_day['year_month'].value_counts().sort_index()
print(count_per_month)

In [ ]:
fig = px.line(
    df_day,
    x='date',
    y='PM_2.5',
    markers=True,
    title="Daily Mean PM₂.₅ Concentration",
    labels={
        'mean_v': 'Concentration (µg/m³)',
        'date': 'Date'}
)
fig.show()

### Time series analysis

In [ ]:
serie=df_day['PM_2.5']
# Create a figure with 2 subplots
fig, axes = plt.subplots(2, 1, figsize=(10, 8))
# Plot autocorrelation (ACF)
plot_acf(serie, ax=axes[0], lags=100)
axes[0].set_title('Autocorrelation (ACF)')
# Plot partial autocorrelation (PACF)
plot_pacf(serie, ax=axes[1], lags=100)
axes[1].set_title('Partial Autocorrelation (PACF)')
plt.show()

In [ ]:
decomposition = seasonal_decompose(df_day['PM_2.5'], model='additive', period=7)
trend = decomposition.trend
seasonal = decomposition.seasonal
resid = decomposition.resid
fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# Original series
axs[0].plot(df_day['PM_2.5'], label='Original')
axs[0].legend(loc='upper left')
axs[0].set_title('Original Series')

# Trend
axs[1].plot(trend, label='Trend', color='orange')
axs[1].legend(loc='upper left')
axs[1].set_title('Trend')

# Seasonal
axs[2].plot(seasonal, label='Seasonality', color='green')
axs[2].legend(loc='upper left')
axs[2].set_title('Seasonality')

# Residuals
axs[3].plot(resid, label='Residuals', color='red')
axs[3].legend(loc='upper left')
axs[3].set_title('Residuals')

plt.tight_layout()
plt.show()


In [ ]:
decomposition = seasonal_decompose(df_day['PM_2.5'], model='additive', period=30)

trend = decomposition.trend
seasonal = decomposition.seasonal
resid = decomposition.resid
fig, axs = plt.subplots(4, 1, figsize=(12, 10), sharex=True)

# Original series
axs[0].plot(df_day['PM_2.5'], label='Original')
axs[0].legend(loc='upper left')
axs[0].set_title('Original Series')

# Trend
axs[1].plot(trend, label='Trend', color='orange')
axs[1].legend(loc='upper left')
axs[1].set_title('Trend')

# Seasonal
axs[2].plot(seasonal, label='Seasonality', color='green')
axs[2].legend(loc='upper left')
axs[2].set_title('Seasonality')

# Residuals
axs[3].plot(resid, label='Residuals', color='red')
axs[3].legend(loc='upper left')
axs[3].set_title('Residuals')

plt.tight_layout()
plt.show()


In [ ]:
# Decompositions with different periods
decomp_7 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=7)
decomp_30 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=30)

# Create the figure
fig, axs = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Seasonal component with period = 7
axs[0].plot(decomp_7.seasonal, label='Seasonality (P=7)', color='green')
axs[0].legend(loc='upper left')
axs[0].set_title('Seasonality (period = 7 days)')

# Seasonal component with period = 30
axs[1].plot(decomp_30.seasonal, label='Seasonality (P=30)', color='blue')
axs[1].legend(loc='upper left')
axs[1].set_title('Seasonality (period = 30 days)')

plt.tight_layout()
plt.show()


In [ ]:
# Decompositions with different periods
decomp_7 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=7)
decomp_30 = seasonal_decompose(df_day['PM_2.5'], model='additive', period=30)

# Create the figure
fig, axs = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

# Trend component with period = 7
axs[0].plot(decomp_7.trend, label='Trend (P=7)', color='orange')
axs[0].legend(loc='upper left')
axs[0].set_title('Trend (period = 7 days)')

# Trend component with period = 30
axs[1].plot(decomp_30.trend, label='Trend (P=30)', color='red')
axs[1].legend(loc='upper left')
axs[1].set_title('Trend (period = 30 days)')

plt.tight_layout()
plt.show()


In [ ]:
print("Mean of residuals:", resid.mean())

In [ ]:
plot_acf(resid.dropna(), lags=30)
plt.title("ACF of residuals")
plt.show()

We study the stationarity of a time series to determine whether its statistical properties — such as mean, variance, and autocorrelation — remain constant over time or change.

In [ ]:
def test_stationarity(series, max_diff=2, verbose=True):
    for d in range(max_diff + 1):
        if d > 0:
            series = series.diff().dropna()
        result = adfuller(series)
        pvalue = result[1]
        
        if verbose:
            print(f"\nDifferencing order: {d}")
            print(f"ADF Statistic: {result[0]:.4f}")
            print(f"p-value: {pvalue:.4f}")
            print("=> Stationary ✅" if pvalue < 0.05 else "=> Not stationary ❌")
        
        if pvalue < 0.05:
            return d  
    
    return None  

stationarity_order = test_stationarity(df_day['PM_2.5'])


In [ ]:
df_day["day_week"] = df_day["date"].dt.day_name()
mean_day = df_day.groupby('day_week',observed=False)['PM_2.5'].mean()
mean_day = mean_day.reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])
print(mean_day)

In [ ]:
plt.figure(figsize=(8,5))
mean_day.plot(kind='bar', color='skyblue')
plt.title('Mean of the pollution PM2.5 by day of the week')
plt.ylabel('PM2.5 moyen')
plt.xlabel('Day week')
plt.show()

In [ ]:
# Ensure month column is integer type and sort values properly
df['month'] = df['month'].astype(int)
# Create the mean PM2.5 by month
mean_month = df.groupby('month')['PM_2.5'].mean().reindex(range(1, 13))
# Define labels
months_labels = ['January', 'February', 'March', 'April', 'May', 'June',
                 'July', 'August', 'September', 'October', 'November', 'December']
# Plotting
plt.figure(figsize=(10, 6))
mean_month.plot(kind='bar', color='coral')
plt.title('Mean PM2.5 Pollution by Month')
plt.ylabel('Mean PM2.5')
plt.xlabel('Month')
plt.xticks(ticks=range(12), labels=months_labels, rotation=45)
plt.tight_layout()
plt.show()